In [5]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
from collections import OrderedDict

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.nn import DataParallel

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/media/rene/data')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))

# import my functions:
from utils import make_cfiar10
from functions import*
from models import*
sys.path.append('/media/rene/code/WideResNet-pytorch')
from wideresnet import WideResNet

device=torch.device("cpu")
print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/code/idc/src
2
True
0


In [50]:
# epochs1, epochs2 = int(epochs1), int(epochs2)
num_workers = 4

PATH = Path('/media/rene/data/')
save_path = Path('/media/rene/code/WideResNet-pytorch/runs')
model_name_list = ['WideResNet-28-10_0/model_best.pth.tar', 'WideResNet-28-10_1/model_best.pth.tar', 'WideResNet-28-10_2/model_best.pth.tar', 
                   'WideResNet-28-10_3/model_best.pth.tar', 'WideResNet-28-10_4/model_best.pth.tar', 'WideResNet-28-10_5/model_best.pth.tar']
batch_size = 128

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid', transformation=transform_test)

pretrained_model_list = []
# First trained model was with DATA PARALLEL
model = WideResNet(28, 10, 20)
model = model.to(device)

state_dict = torch.load(os.path.join(save_path, 'WideResNet-28-10_0/model_best.pth.tar'))['state_dict']

# create new OrderedDict that does not contain `module.`
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)
pretrained_model_list.append(model)

# get all the models
for i, model_name in enumerate(model_name_list[1:4]):
    print('------------loading model: ', model_name)
    model = WideResNet(28, 10, 20)
    model = model.to(device)

    # original saved file with DataParallel
    state_dict = torch.load(os.path.join(save_path, model_name))['state_dict']
    model.load_state_dict(state_dict)
    pretrained_model_list.append(model)

model = Fusion2More(pretrained_model_list)#, 40, 10)

######################  TRAIN LAST FEW LAYERS
print('training last few layers')

model_name = 'fusionWRN_more_last4_1'
for p in model.parameters():
    p.requires_grad = True
for p in model.model1.parameters():
    p.requires_grad = False
for p in model.model2.parameters():
    p.requires_grad = False
for p in model.model3.parameters():
    p.requires_grad = False
for p in model.model4.parameters():
    p.requires_grad = False

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(filter(lambda p: p.requires_grad,model.parameters()), lr=.005, momentum=0.9, weight_decay=5e-4)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=int(epochs1/4), gamma=0.2)

# best_acc, model = train_model(model, criterion, optimizer, scheduler, epochs1, 
#                            dataloaders, dataset_sizes, device=device)
# torch.save(model.state_dict(), str(save_path / model_name))

Files already downloaded and verified
Files already downloaded and verified
------------loading model:  WideResNet-28-10_1/model_best.pth.tar
------------loading model:  WideResNet-28-10_2/model_best.pth.tar
------------loading model:  WideResNet-28-10_3/model_best.pth.tar
training last few layers


In [53]:
for i, child in enumerate(model.model4.children()):
    for i, grandchild in enumerate(child.children()):
        print('------', i)
        print(grandchild)



------ 0
Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
------ 1
NetworkBlock(
  (layer): Sequential(
    (0): BasicBlock(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(16, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace)
      (conv2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (convShortcut): Conv2d(16, 320, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (bn1): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (conv1): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [48]:
print(type(model.model4))
print(model.model4.children())
for i, child in enumerate(model.model4.children()):
    if(i >=3):
        for p in child.parameters():
            p.requires_grad = False


# print(model.model4.block3)
# print(model.model4.block3)
# print(model.model4.block3)

<class 'wideresnet.WideResNet'>
<generator object Module.children at 0x7f5980348a98>


In [16]:

for p in model.block3.parameters():
    print(p)

Parameter containing:
tensor([[[[ 1.7888e-08, -1.4958e-07, -3.9252e-08],
          [ 7.3968e-09, -3.3649e-07, -1.0936e-07],
          [-1.8904e-08, -1.2209e-07, -5.1257e-08]],

         [[ 7.0011e-08, -2.7621e-08, -6.1985e-09],
          [ 4.7053e-08, -2.5814e-08,  5.4354e-08],
          [ 5.2845e-09, -8.3355e-08,  6.3604e-08]],

         [[-4.3028e-08, -1.9149e-07, -9.9852e-08],
          [-3.1629e-07, -2.2304e-07, -1.3026e-08],
          [-1.7752e-07, -1.7889e-07,  1.2664e-07]]],


        [[[ 8.8480e-03, -8.0085e-02, -4.1302e-02],
          [-7.9808e-02, -1.7260e-01, -1.7430e-01],
          [-4.4792e-02, -1.7985e-01, -1.3897e-01]],

         [[ 2.9839e-02,  3.0404e-02,  1.9140e-02],
          [ 2.3686e-02,  1.0464e-01,  1.8753e-02],
          [ 3.1975e-03, -1.1382e-03, -1.3681e-02]],

         [[-5.1155e-02,  6.4777e-02,  3.1901e-02],
          [ 4.0541e-02,  2.9163e-01,  1.5389e-01],
          [ 1.5941e-02,  1.6370e-01,  7.6523e-02]]],


        [[[-6.4256e-02,  7.0170e-02,  6.8063

Parameter containing:
tensor([[[[ 1.8727e-03,  3.4164e-03,  3.6470e-03],
          [ 1.1090e-03,  1.7259e-03,  1.5304e-03],
          [-2.7377e-04, -2.2082e-05,  2.1265e-05]],

         [[ 1.2161e-03,  6.7341e-04, -5.4687e-04],
          [ 3.1827e-04,  7.1545e-04,  7.1823e-04],
          [-8.0349e-05, -1.2446e-04,  1.4167e-03]],

         [[ 1.9206e-04,  2.4028e-04,  1.4823e-03],
          [-1.3127e-03, -8.5375e-04,  3.6384e-03],
          [-7.7943e-04,  1.4092e-03,  3.8072e-03]],

         ...,

         [[ 3.6381e-04, -1.2445e-03, -5.2500e-05],
          [ 2.9727e-04, -6.7708e-04,  4.1057e-05],
          [ 3.7795e-04, -8.9576e-04, -6.4868e-04]],

         [[ 3.2350e-04,  6.1042e-05, -1.7524e-04],
          [ 5.2689e-04, -2.3724e-04, -2.3797e-04],
          [ 2.0714e-04, -8.9442e-05,  1.9470e-05]],

         [[-3.5748e-05, -2.3427e-05,  1.7181e-05],
          [ 9.9301e-06, -7.4073e-05, -5.7777e-05],
          [ 2.3601e-05, -2.6816e-05, -5.8210e-05]]],


        [[[-7.0106e-04, -1.2880

Parameter containing:
tensor([[[[ 4.3088e-03, -6.7838e-03, -1.2460e-02],
          [ 6.8089e-03, -1.8823e-03, -1.2239e-02],
          [ 3.9217e-03,  4.3769e-03, -8.1921e-03]],

         [[ 8.6252e-03, -1.4674e-03, -2.9517e-03],
          [ 3.8254e-03,  3.8498e-03,  3.5260e-03],
          [-2.2897e-03,  3.6953e-03,  4.3180e-03]],

         [[ 6.8508e-03,  3.4041e-03,  6.4482e-03],
          [ 4.9938e-03,  1.5740e-03,  4.2592e-03],
          [ 2.4391e-03,  1.7136e-03,  4.0798e-03]],

         ...,

         [[ 4.0892e-03,  1.3875e-03, -4.8075e-04],
          [ 4.4174e-03,  3.1885e-03, -7.7612e-04],
          [ 3.7711e-03,  2.3193e-03, -1.0798e-04]],

         [[-1.1337e-04,  5.1747e-05,  4.2213e-04],
          [ 9.3320e-04, -1.3701e-03, -8.5438e-04],
          [ 1.5094e-03, -1.1408e-03, -2.4054e-03]],

         [[-2.9392e-03, -2.7894e-03, -3.6100e-03],
          [ 9.8615e-04, -1.1015e-03, -4.7971e-03],
          [ 2.8747e-03,  1.7414e-04, -3.7137e-03]]],


        [[[-3.3431e-03,  1.4502

Parameter containing:
tensor([[[[-9.4926e-04, -2.0896e-03, -2.2612e-03],
          [ 4.0030e-04, -1.0170e-03, -7.5973e-04],
          [ 1.3924e-03,  2.8601e-04,  3.1990e-04]],

         [[-1.2391e-03, -1.7446e-03, -8.4055e-04],
          [-1.6265e-03, -1.2777e-03,  4.2032e-04],
          [-6.7500e-04,  7.6217e-04,  2.2416e-03]],

         [[-1.7052e-03, -1.4688e-03, -6.7252e-04],
          [ 9.4896e-04,  7.2761e-04, -1.8864e-03],
          [ 1.2415e-03,  1.3918e-03, -2.0757e-03]],

         ...,

         [[-5.0253e-04, -3.8523e-04, -7.2275e-04],
          [ 2.7585e-04, -1.2108e-03, -1.5737e-03],
          [ 8.1521e-04, -1.7259e-03, -3.2155e-03]],

         [[ 9.1171e-05,  2.4542e-04, -3.1136e-05],
          [-4.8042e-04,  4.1353e-04, -7.7007e-04],
          [ 6.0337e-04,  9.3717e-04, -3.0816e-04]],

         [[-2.5956e-03, -1.9282e-03, -2.0531e-03],
          [-3.0450e-03, -1.6195e-03, -1.4760e-03],
          [-2.8987e-03, -1.6307e-03, -8.0967e-04]]],


        [[[ 2.4057e-03,  9.7706

Parameter containing:
tensor([[[[-4.7621e-04, -4.6405e-04,  1.2879e-03],
          [ 8.1358e-04,  3.3344e-04, -1.0858e-03],
          [ 1.4033e-03,  2.6025e-04, -1.2808e-03]],

         [[ 5.4179e-04, -2.2926e-03, -2.1779e-03],
          [-1.2499e-03, -1.6235e-03, -2.8461e-03],
          [-2.1731e-03, -4.9903e-04, -7.5923e-04]],

         [[-2.8740e-03, -2.2301e-03, -7.2440e-05],
          [-2.1351e-03, -2.2782e-03,  6.7025e-04],
          [-3.9088e-04, -5.8597e-04,  3.1038e-03]],

         ...,

         [[-2.1752e-03,  3.6495e-04,  1.4454e-03],
          [-1.9139e-03, -5.3666e-04,  3.4274e-04],
          [ 4.6136e-04, -2.0574e-03,  1.0041e-03]],

         [[ 9.8774e-04,  2.7514e-04,  5.1024e-05],
          [ 5.7127e-04,  5.1500e-04, -2.6930e-04],
          [-1.1446e-03, -1.0783e-03, -8.4503e-04]],

         [[ 5.6261e-04, -2.8749e-04, -7.4151e-04],
          [-1.3291e-03, -2.8933e-03, -2.2683e-03],
          [-1.7074e-03, -3.1817e-03, -1.5446e-03]]],


        [[[-1.5254e-03,  6.2171

Parameter containing:
tensor([[[[-2.5839e-04, -5.1135e-05, -1.0294e-04],
          [-4.5336e-04, -1.2067e-04, -2.3906e-04],
          [-1.0192e-03, -9.0534e-04, -6.9199e-04]],

         [[-2.2216e-03, -1.1496e-03,  5.2650e-05],
          [-1.4705e-03, -1.0828e-03,  1.6368e-04],
          [-2.9201e-03, -2.6760e-03, -3.6167e-04]],

         [[ 7.4130e-05,  4.4854e-04, -1.5857e-03],
          [ 2.1021e-03,  9.1472e-04, -5.8468e-04],
          [ 2.7029e-03,  1.1667e-03,  1.2452e-03]],

         ...,

         [[-3.7031e-04, -9.5916e-04, -4.7430e-03],
          [-2.7866e-03, -2.2374e-03, -1.9191e-03],
          [-2.5826e-03,  5.2881e-05,  3.0736e-03]],

         [[ 2.6473e-03,  4.2860e-03,  3.0586e-04],
          [ 1.1120e-03, -2.5217e-03, -2.6760e-03],
          [-4.5177e-04,  6.9748e-04,  3.0395e-04]],

         [[-1.9454e-04, -1.6205e-03, -8.6807e-04],
          [-1.0506e-03, -2.3746e-03, -2.0059e-03],
          [ 4.0071e-04, -8.5448e-04, -1.8922e-03]]],


        [[[-2.1717e-04, -1.9912

Parameter containing:
tensor([[[[ 3.2261e-03, -3.4484e-03, -8.9731e-04],
          [ 5.3694e-03,  3.8724e-03,  2.4501e-03],
          [-1.7513e-03,  3.3196e-05,  1.5830e-03]],

         [[ 1.6199e-03,  6.7223e-03,  4.7385e-04],
          [ 3.0951e-03,  1.5893e-03,  3.0403e-03],
          [ 1.8987e-03,  1.7898e-03,  4.1413e-03]],

         [[ 9.8784e-04,  1.8940e-03, -2.1547e-03],
          [-3.6844e-04,  3.0867e-05, -1.2548e-03],
          [-1.5579e-03, -2.8996e-04, -4.1307e-04]],

         ...,

         [[-1.8487e-03, -4.3107e-03, -2.7520e-03],
          [-1.1196e-03,  4.2727e-04, -4.3998e-03],
          [-6.7964e-04,  1.5474e-03, -1.3953e-03]],

         [[-3.3112e-03, -9.9786e-03, -8.3986e-03],
          [-1.3931e-03, -1.0531e-02, -8.2518e-03],
          [ 1.2549e-03, -2.9441e-03, -5.7893e-03]],

         [[-5.0992e-03, -4.1156e-03, -5.6881e-03],
          [-3.6163e-03, -2.3789e-03, -5.2962e-03],
          [-1.3193e-04, -2.3236e-03, -4.0914e-03]]],


        [[[ 2.0884e-03, -3.5038

Parameter containing:
tensor([[[[ 1.8399e-04,  3.3751e-04, -3.7313e-04],
          [-2.9674e-03, -2.1075e-03, -1.5908e-03],
          [-2.6958e-03, -2.6134e-03, -6.2418e-04]],

         [[ 6.9529e-04,  4.3733e-04, -2.4509e-04],
          [ 1.4883e-03,  9.2899e-04,  2.2751e-04],
          [ 8.3660e-04,  5.7551e-04,  4.7113e-04]],

         [[ 4.9855e-04,  2.8931e-03, -7.4567e-04],
          [-7.6728e-04, -5.5886e-04, -2.3659e-03],
          [-1.5412e-03, -2.3232e-03, -4.9994e-04]],

         ...,

         [[-1.7256e-03, -9.0131e-04,  1.0530e-03],
          [-1.3970e-03, -7.8355e-04,  1.2021e-03],
          [ 1.0544e-03,  1.9237e-03,  2.6351e-03]],

         [[-1.4306e-04,  1.4257e-04,  9.9346e-05],
          [-1.6045e-04, -5.5823e-05, -2.1730e-04],
          [ 1.2575e-05, -3.3095e-05, -1.3920e-04]],

         [[-1.3330e-04, -2.5131e-03,  2.3328e-04],
          [ 8.8989e-04, -1.5038e-03,  1.9293e-04],
          [ 3.5393e-03,  2.6606e-03,  2.7274e-03]]],


        [[[ 2.8095e-03,  3.0888

Parameter containing:
tensor([[[[ 2.6724e-04, -5.3601e-04, -8.3545e-05],
          [-7.0852e-05, -3.7707e-04, -3.5201e-04],
          [-4.9977e-04, -7.6157e-04, -2.5642e-04]],

         [[ 2.6982e-05,  8.1326e-06,  1.2307e-06],
          [ 1.6803e-05,  1.7569e-05,  2.0671e-05],
          [ 1.6263e-05,  2.8303e-05,  4.6814e-05]],

         [[ 1.5139e-03, -1.0787e-03, -8.3924e-04],
          [-1.5097e-03, -2.3608e-03, -7.4402e-04],
          [ 2.6754e-03,  2.0220e-03,  1.3407e-03]],

         ...,

         [[-2.2022e-03, -6.2203e-04, -2.2055e-03],
          [-2.0341e-03,  1.5382e-03,  1.0658e-03],
          [ 7.0842e-04,  3.4086e-03,  1.9051e-03]],

         [[-2.9781e-04, -2.2999e-04,  8.1481e-05],
          [ 1.5513e-03,  7.0754e-04,  1.3292e-03],
          [-7.8047e-05,  3.9383e-04,  1.0703e-03]],

         [[-1.1176e-03, -2.2917e-04,  1.1490e-03],
          [ 6.2244e-05,  1.3513e-03,  2.3968e-03],
          [-6.0747e-05,  1.5844e-03,  1.7905e-03]]],


        [[[-2.7154e-05, -2.9784

Parameter containing:
tensor([[[[ 6.3600e-04,  6.6439e-04,  7.6558e-04],
          [ 2.5611e-04,  2.4000e-04,  3.3209e-04],
          [ 6.4311e-05,  8.5861e-05,  3.1149e-05]],

         [[ 1.3329e-03,  1.3059e-03,  9.3650e-04],
          [ 5.7701e-04,  4.0186e-04,  1.1015e-04],
          [ 1.5485e-04,  8.7695e-05, -1.3933e-04]],

         [[-5.8376e-05, -8.5614e-05,  1.7165e-05],
          [-5.2253e-05, -4.1977e-05,  6.1813e-05],
          [ 1.3070e-04,  7.4405e-05,  4.8574e-05]],

         ...,

         [[-5.6503e-05, -3.0320e-05,  2.4806e-06],
          [ 4.0088e-05,  8.4151e-05,  7.2746e-05],
          [ 7.9542e-05,  1.1515e-04,  1.2322e-04]],

         [[-1.7528e-04, -1.4401e-04, -1.7618e-04],
          [-3.2727e-05, -3.6420e-05, -1.5777e-05],
          [-1.6676e-04, -1.1038e-04,  2.5748e-05]],

         [[ 5.3137e-05,  7.4381e-05,  1.8700e-04],
          [ 8.9520e-06,  4.8121e-06,  7.4238e-05],
          [-3.0883e-05, -5.5731e-05, -2.3736e-05]]],


        [[[ 4.4969e-04, -1.0681

In [7]:
########################   TRAIN ALL LAYERS
model.load_state_dict(torch.load(save_path / 'fusionWRN_more_last4_1'))
model_name = 'fusionWRN_more_last4_2'

batch_size = 1
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid')

for p in model.parameters():
    p.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad,model.parameters()), lr=.0001, momentum=0.9, weight_decay=5e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=int(epochs2/3), gamma=0.1)

best_acc, model = train_model(model, criterion, optimizer, scheduler, epochs2, 
                           dataloaders, dataset_sizes, device=device, multi_gpu=True)

torch.save(model.state_dict(), str(save_path / model_name))

FileNotFoundError: [Errno 2] No such file or directory: '/media/rene/code/WideResNet-pytorch/runs/fusionWRN_more_last4_1'